# To explore the `fit` method of class `GBDT`
Firstly, explore the function of binary classification

In [1]:
# open('data/credit.data.csv').read()

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from random import sample
from gbdt.data import DataSet
from gbdt.model import GBDT
from gbdt.model import BinomialDeviance
from gbdt.tree import construct_decision_tree, Tree
from gbdt.tree import MSE

In [4]:
dataset = DataSet('data/credit.data.csv')
dataset.get_label_size()

2

#### The initial properties

In [5]:
max_iter = 20
sample_rate = 0.8
learn_rate = 0.5
max_depth = 7
loss_type = 'binary-classification'
split_points = 0

trees = dict()

#### Loss function: Binary classification

In [6]:
loss = BinomialDeviance(n_classes=dataset.get_label_size())

In [7]:
f = dict()

In [8]:
loss.initialize(f, dataset)
len(f.keys())
# f: f is a dict, and it has 653 key-value pairs. Keys are integers from 1 to 653, and all the values are set to be 0

653

In [9]:
train_data = dataset.get_instances_idset()
# train_data is a set, which contains IDs from 1 to 653

Binary classification:

The residual is $Residual_i = \frac{2y_i}{1+exp(2y_if_i)} $. (Log loss deviance)


Why loss is as above? Refer to 
- [Scikit Binomial Deviance Loss Function](https://stats.stackexchange.com/questions/157870/scikit-binomial-deviance-loss-function)
- [How to derive bernoulli deviance](https://stats.stackexchange.com/questions/208331/how-to-derive-bernoulli-deviance)
- It is called binomial negative log-likelihood loss:
https://pdfs.semanticscholar.org/7efc/245d8ad4cbd6489e3dca6688264bf4f83579.pdf
- in ESLII, it is called __binomial deviance__ or __binomial negative log-likelihood__ P346 
- [GBDT模型](https://www.jianshu.com/p/0bc32c8e4ca8)
- [GBDT训练分类器时，残差是如何计算的？](https://blog.csdn.net/mmc2015/article/details/52398488)
- Friedman's paper: [Greedy Function Approximation: A Gradient Boosting Machine](http://docs.salford-systems.com/GreedyFuncApproxSS.pdf)

The negative binomial log-likelihood loss: $L(y, F) = \log(1+ \exp(-2yF)),~~~ y\in\{-1,1\}$. 

There is an coefficient of 2, because here the label is -1 and 1. 

If the label is 0 and 1, $L(y, F) = \log(1+ \exp(-yF)),~~~ y\in\{0,1\}$

这两个公式本质上是一样的，只是因为label的不同而不同，相互转化的过程参考ESLII的P346即可。因为这里的label是-1,1，所以我们必须使用前一种损失函数！

In [10]:
iter = 1 # for iter in range(1, max_iter+1):
subset = train_data
print('the size of initial subset is', len(subset))
if 0 < sample_rate < 1:
    subset = sample(subset, int(len(subset)*sample_rate))
    print('the size of sampled subset is', len(subset), ', only 80% of the initial size')
residual = loss.compute_residual(dataset, subset, f) # residual is a dict, in which keys are the sampled IDs 
print(len(residual))

leaf_nodes = []
targets = residual


the size of initial subset is 653
the size of sampled subset is 522 , only 80% of the initial size
522


In [11]:
# tree = construct_decision_tree(dataset, subset, targets, 0, leaf_nodes, max_depth, loss, split_points)

In [12]:
# tree.conditionValue
# tree.describe()
# # tree.leafNode
# tree.real_value_feature
# tree.split_feature

Now explore what is exactly function `construct_decision_tree` is:

`depth =  0`

`if depth < max_depth`:

In [13]:
attributes = dataset.get_attributes()
attributes

('A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'A10',
 'A11',
 'A12',
 'A13',
 'A14',
 'A15')

In [14]:
mse = -1
selectedAttibute = None
conditionValue = None
selectedLeftIdSet = []
selectedRightIdSet = []

In [15]:
for attribute in attributes:
    is_real_type = dataset.is_real_type_field(attribute)
    attrValues = dataset.get_distinct_valueset(attribute)
    if is_real_type and split_points > 0 and len(attrValues) > split_points: # Here is False, no next line
        attrValues = sample(attrValues, split_points)
    for attrValue in attrValues:
        leftIdSet = []
        rightIdSet = []    
        for Id in subset:
            instance = dataset.get_instance(Id) # a dict
            value = instance[attribute]
            # 计算量巨大：如果这个attribute有1000种不同取值，subset中有500个Id，则下面的判断条件要运行5*10^5遍
            # 枚举此attribute的每个取值，进行节点的分裂，找到loss最小的那个
            if (is_real_type and value< attrValue) or (not is_real_type and value == attrValue):
                leftIdSet.append(Id)
            else:
                rightIdSet.append(Id)
        leftTargets = [targets[id] for id in leftIdSet] # 梯度
        rightTargets = [targets[id] for id in rightIdSet]
        sum_mse = MSE(leftTargets)+MSE(rightTargets)
#         print(sum_mse)
        if mse < 0 or sum_mse < mse:
            selectedAttribute = attribute
            conditionValue = attrValue
            mse = sum_mse
            selectedLeftIdSet = leftIdSet
            selectedRightIdSet = rightIdSet
#     print('Is real type:', is_real_type, '   Attribute values:', attrValues)
#     print(mse)
#     print(leaf_nodes)

In [16]:
tree = Tree()
tree.split_feature = selectedAttribute
tree.real_value_feature = dataset.is_real_type_field(selectedAttribute)
tree.conditionValue = conditionValue
tree.leftTree = construct_decision_tree(dataset, selectedLeftIdSet, targets, 1, leaf_nodes, max_depth, loss)
tree.rightTree = construct_decision_tree(dataset, selectedRightIdSet, targets, 1, leaf_nodes, max_depth, loss)
# 经过construct_decision_tree这个递归函数之后，Id最后都在leaf_nodes中

In [17]:
# the subset Id are all in the leaf nodes eventually
listlen = [i.idset for i in leaf_nodes]
sum([len(i) for i in listlen])

522

In [18]:
loss.update_f_value(f, tree, leaf_nodes, subset, dataset, learn_rate)